<a href="https://colab.research.google.com/github/Lmdela/Trabajo-de-Mineria-de-Datos/blob/Trabajo-Mineria/Adecuaci%C3%B3n_de_la_base_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Adecuación de la base de datos para hacer Redes LSTM para la predicción de la temperatura del mar**

**Autores**
1. Antony D'Acunto, adacuntob@unal.edu.co
2. Luisa de la Hortúa, lmdela@unal.edu.co

**Fork**

**Referencias**
1. Curso de Minería de Datos Universidad Nacional de Colombia. Profesor: Alvaro Montenegro. https://github.com/AprendizajeProfundo/Ciencia-de-Datos
2. https://calcofi.org/ccdata/database.html
5. https://new.data.calcofi.org/index.php/database/calcofi-database/bottle-field-descriptions
4. http://colah.github.io/posts/2015-08-Understanding-LSTMs/



# **Problema principal**

El manejo adecuado de los datos es importante para hacer un correcto uso de las redes LSTM. Por lo que el primer problema que se tiene que resolver es la escogencia de las variables involucradas en la red. En este caso, se tiene que separar una variable en substrings, porque contiene una gran cantidad de información. 

* En este caso escogimos Depth_Id porque contiene la información de la fecha,la temperatura, la profundidad, la densidad del agua y la salinidad del mar. 

En el primer paso se está haciendo la importación de librerias necesarias para el manejo de los datos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 

Se hace la lectura de la base de datos que vamos a usar, en este caso es una base llamada "Bottles" que puede ser encontrada en Kaggle en el primer link en las referencias.

In [ ]:
dataframe = pd.read_csv(r"194903-201907_Bottle.csv", sep=(","), engine="python")
pd.set_option('display.max_rows', 10)
dataframe

,Cst_Cnt,Btl_Cnt,Sta_ID,Depth_ID,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_�mol/Kg,BtlNum,RecInd,T_prec,T_qual,S_prec,S_qual,P_qual,O_qual,SThtaq,O2Satq,ChlorA,Chlqua,Phaeop,Phaqua,PO4uM,PO4q,SiO3uM,SiO3qu,NO2uM,NO2q,NO3uM,NO3q,NH3uM,NH3q,C14As1,C14A1p,C14A1q,C14As2,C14A2p,C14A2q,DarkAs,DarkAp,DarkAq,MeanAs,MeanAp,MeanAq,IncTim,LightP,R_Depth,R_TEMP,R_Sal,R_DYNHT,R_Nuts,R_Oxy_�mol/Kg,DIC1,DIC2,TA1,TA2,pH1,pH2,DIC Quality Comment
0,1,1,054.0 056.0,19-4903CR-HY-060-0930-05400560-0000A-3,0,10.500,33.4400,NaN,25.64900,NaN,NaN,NaN,3,1.0,NaN,2.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,0.0,10.50,233.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,054.0 056.0,19-4903CR-HY-060-0930-05400560-0008A-3,8,10.460,33.4400,NaN,25.65600,NaN,NaN,NaN,3,2.0,NaN,2.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,8.0,10.46,232.5,0.010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,054.0 056.0,19-4903CR-HY-060-0930-05400560-0010A-7,10,10.460,33.4370,NaN,25.65400,NaN,NaN,NaN,7,2.0,NaN,3.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,10.0,10.46,232.8,0.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,054.0 056.0,19-4903CR-HY-060-0930-05400560-0019A-3,19,10.450,33.4200,NaN,25.64300,NaN,NaN,NaN,3,2.0,NaN,2.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,19.0,10.45,234.1,0.040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,054.0 056.0,19-4903CR-HY-060-0930-05400560-0020A-7,20,10.450,33.4210,NaN,25.64300,NaN,NaN,NaN,7,2.0,NaN,3.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,20.0,10.45,234.0,0.040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885117,35196,885118,093.3 110.0,20-1907BH-MX-196-0039-09331100-0514A-3,514,6.036,34.2735,0.350,26.98561,5.03,15.22,1.0,3,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,9.0,3.13,NaN,76.44,NaN,0.000,8.0,40.18,NaN,0.00,4.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,518.0,5.99,114.0,1.045,0.00,15.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
885118,35197,885119,093.4 026.4,20-1907BH-MX-192-2208-09340264-0000A-7,0,20.931,33.7543,5.918,23.56583,115.69,258.21,NaN,7,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.482,NaN,0.169,NaN,0.33,NaN,1.86,NaN,0.018,NaN,0.10,NaN,0.22,NaN,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,0.0,20.93,431.5,0.000,0.22,258.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
885119,35197,885120,093.4 026.4,20-1907BH-MX-192-2208-09340264-0002A-3,2,20.931,33.7543,5.918,23.56604,115.69,258.21,3.0,3,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.482,NaN,0.169,NaN,0.33,NaN,1.86,NaN,0.000,8.0,0.10,NaN,0.22,NaN,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,2.0,20.93,431.5,0.009,0.22,258.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
885120,35197,885121,093.4 026.4,20-1907BH-MX-192-2208-09340264-0005A-3,5,20.558,33.7410,5.910,23.65613,114.74,257.84,2.0,3,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.528,NaN,0.157,NaN,0.21,NaN,1.80,NaN,0.000,8.0,0.00,8.0,0.11,NaN,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,5.0,20.56,423.1,0.021,0.11,257.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En este paso, se hace la adecuación de los datos para ser estructurados y manejados de la mejor manera. En donde se va a separar la variable "Depth_ID" en sus diferentes componentes, de las cuales solo vamos a hacer uso del año y  mes. 

In [ ]:
year=np.empty(len(dataframe.Depth_ID))
month=np.empty(len(dataframe.Depth_ID))
for x in range(0,len(dataframe.Depth_ID)):
    year[x]=int(dataframe.Depth_ID[x][0:2])*100+int(dataframe.Depth_ID[x][3:5])
    month[x]=int(dataframe.Depth_ID[x][5:7])

In [ ]:
df=pd.DataFrame(np.column_stack((year,month)),columns=['Year','Month'])
df

,Year,Month
0,1949.0,3.0
1,1949.0,3.0
2,1949.0,3.0
3,1949.0,3.0
4,1949.0,3.0
...,...,...
885117,2019.0,7.0
885118,2019.0,7.0
885119,2019.0,7.0
885120,2019.0,7.0


Se seleccionan las variables que consideramos para hacer la red LSTM, en las cuales podemos ver la profundidad y la salinidad, la densidad potencial del agua.

In [ ]:
data = dataframe[['Depthm','T_degC','Salnty','STheta']]
data=pd.concat([data, df], axis=1)
data


,Depthm,T_degC,Salnty,STheta,Year,Month
0,0,10.500,33.4400,25.64900,1949.0,3.0
1,8,10.460,33.4400,25.65600,1949.0,3.0
2,10,10.460,33.4370,25.65400,1949.0,3.0
3,19,10.450,33.4200,25.64300,1949.0,3.0
4,20,10.450,33.4210,25.64300,1949.0,3.0
...,...,...,...,...,...,...
885117,514,6.036,34.2735,26.98561,2019.0,7.0
885118,0,20.931,33.7543,23.56583,2019.0,7.0
885119,2,20.931,33.7543,23.56604,2019.0,7.0
885120,5,20.558,33.7410,23.65613,2019.0,7.0


Para poder hacer el correctamente las redes necesitamos tener un criterio para evaluar la temperatura del mar, por lo tanto se vio que la única variable que no cambia con respecto al tiempo es la profundidad. De donde decidimos separarla en tres profundidades: 0-50m va a ser considerada una profundidad baja, de 50-200 profundidad media y de 200-hasta el maximo valor que se pueda tomar va a ser considerado profundidad alta. La segmentación anterior es tomada ya que los datos por debajo de una profundidad de 1000 son muy pocos y tienen muchos datos faltantes. Se intento adecuar lo más posible a la siguiente imagen:![texto alternativo](http://1.bp.blogspot.com/-U8SP8CaWzk0/VHEaEiw729I/AAAAAAAAAEg/xINzbi8CaJ0/s1600/Diagrama-de-las-capas-del-oc%25C3%25A9ano.jpg)

In [ ]:
low=[None]*len(dataframe.Depth_ID)
mid=[None]*len(dataframe.Depth_ID)
high=[None]*len(dataframe.Depth_ID)
for x in range(0,len(dataframe.Depth_ID)):
    dm=data.Depthm[x]
    if dm<=50:
        low[x]=data.Depthm[x]
    elif dm<=200:
        mid[x]=data.Depthm[x]
    else:
        high[x]=data.Depthm[x]
data=data.drop(['Depthm'], axis=1)

lowdf=pd.concat([data,pd.DataFrame(low,columns=['Depthm'])], axis=1)
lowdf=lowdf.dropna()
middf=pd.concat([data,pd.DataFrame(mid,columns=['Depthm'])], axis=1)
middf=middf.dropna()
highdf=pd.concat([data,pd.DataFrame(high,columns=['Depthm'])], axis=1)
highdf=highdf.dropna()

In [ ]:
pd.set_option('display.max_rows', 10)
lowdf

,T_degC,Salnty,STheta,Year,Month,Depthm
0,10.500,33.4400,25.64900,1949.0,3.0,0.0
1,10.460,33.4400,25.65600,1949.0,3.0,8.0
2,10.460,33.4370,25.65400,1949.0,3.0,10.0
3,10.450,33.4200,25.64300,1949.0,3.0,19.0
4,10.450,33.4210,25.64300,1949.0,3.0,20.0
...,...,...,...,...,...,...
885096,16.814,33.5189,24.42608,2019.0,7.0,50.0
885118,20.931,33.7543,23.56583,2019.0,7.0,0.0
885119,20.931,33.7543,23.56604,2019.0,7.0,2.0
885120,20.558,33.7410,23.65613,2019.0,7.0,5.0


Para poder ver la secuencia que se presenta en dotos tenemos que separar los años en 4 periodos, los cuales van a estar ligados con la estacionalidad en las costas de California. Esta parte del código nos ayuda a poder manejar los datos ausentes que se van a llenar con las redes LSTM. 

In [ ]:
YearF=4
YearT=12/YearF
YearI=int(max(df.Year)-min(df.Year)+1)
Temp=[None]*YearI*YearF
Sal=[None]*YearI*YearF
ST=[None]*YearI*YearF
Yr=[None]*YearI*YearF
Ssn=[None]*YearI*YearF
for x in range(0,YearI):
    for y in range(0,YearF):
        Yr[YearF*x+y]=1949+x
        Ssn[YearF*x+y]=y+1
        if y == 0:
                Sal[YearF*x+y]=(lowdf.Salnty.loc[lowdf.Year == 1949+x-1].loc[lowdf.Month>=12].sum()+lowdf.Salnty.loc[lowdf.Year == 1949+x].loc[lowdf.Month<3].sum())/(len(lowdf.T_degC.loc[lowdf.Year == 1949+x-1].loc[lowdf.Month>=12])+len(lowdf.T_degC.loc[lowdf.Year == 1949+x].loc[lowdf.Month<3]))
                ST[YearF*x+y]=(lowdf.STheta.loc[lowdf.Year == 1949+x-1].loc[lowdf.Month>=12].sum()+lowdf.Salnty.loc[lowdf.Year == 1949+x].loc[lowdf.Month<3].sum())/(len(lowdf.T_degC.loc[lowdf.Year == 1949+x-1].loc[lowdf.Month>=12])+len(lowdf.T_degC.loc[lowdf.Year == 1949+x].loc[lowdf.Month<3]))
                Temp[YearF*x+y]=(lowdf.Salnty.loc[lowdf.Year == 1949+x-1].loc[lowdf.Month>=12].sum()+lowdf.Salnty.loc[lowdf.Year == 1949+x].loc[lowdf.Month<3].sum())/(len(lowdf.T_degC.loc[lowdf.Year == 1949+x-1].loc[lowdf.Month>=12])+len(lowdf.T_degC.loc[lowdf.Year == 1949+x].loc[lowdf.Month<3]))
        else:
                Sal[YearF*x+y]=lowdf.Salnty.loc[lowdf.Year == 1949+x].loc[lowdf.Month>=(y+1)*YearT-1].loc[lowdf.Month<(y+2)*YearT-1].sum()/len(lowdf.T_degC.loc[lowdf.Year == 1949+x].loc[lowdf.Month>=(y+1)*YearT-1].loc[lowdf.Month<(y+2)*YearT-1])
                ST[YearF*x+y]=lowdf.STheta.loc[lowdf.Year == 1949+x].loc[lowdf.Month>=(y+1)*YearT-1].loc[lowdf.Month<(y+2)*YearT-1].sum()/len(lowdf.T_degC.loc[lowdf.Year == 1949+x].loc[lowdf.Month>=(y+1)*YearT-1].loc[lowdf.Month<(y+2)*YearT-1])
                Temp[YearF*x+y]=lowdf.T_degC.loc[lowdf.Year == 1949+x].loc[lowdf.Month>=(y+1)*YearT-1].loc[lowdf.Month<(y+2)*YearT-1].sum()/len(lowdf.T_degC.loc[lowdf.Year == 1949+x].loc[lowdf.Month>=(y+1)*YearT-1].loc[lowdf.Month<(y+2)*YearT-1])
lowdata=pd.DataFrame(np.column_stack((Yr,Ssn,Temp,Sal,ST)),columns=['Year','Season','T_degC','Salnty','STheta'])
lowdata=lowdata.drop(lowdata.index[0])

YearF=4
YearT=12/YearF
YearI=int(max(df.Year)-min(df.Year)+1)
Temp=[None]*YearI*YearF
Sal=[None]*YearI*YearF
ST=[None]*YearI*YearF
Yr=[None]*YearI*YearF
Ssn=[None]*YearI*YearF
for x in range(0,YearI):
    for y in range(0,YearF):
        Yr[YearF*x+y]=1949+x
        Ssn[YearF*x+y]=y+1
        if y == 0:
            Sal[YearF*x+y]=(middf.Salnty.loc[middf.Year == 1949+x-1].loc[middf.Month>=12].sum()+middf.Salnty.loc[middf.Year == 1949+x].loc[middf.Month<3].sum())/(len(middf.T_degC.loc[middf.Year == 1949+x-1].loc[middf.Month>=12])+len(middf.T_degC.loc[middf.Year == 1949+x].loc[middf.Month<3]))
            ST[YearF*x+y]=(middf.STheta.loc[middf.Year == 1949+x-1].loc[middf.Month>=12].sum()+middf.Salnty.loc[middf.Year == 1949+x].loc[middf.Month<3].sum())/(len(middf.T_degC.loc[middf.Year == 1949+x-1].loc[middf.Month>=12])+len(middf.T_degC.loc[middf.Year == 1949+x].loc[middf.Month<3]))
            Temp[YearF*x+y]=(middf.Salnty.loc[middf.Year == 1949+x-1].loc[middf.Month>=12].sum()+middf.Salnty.loc[middf.Year == 1949+x].loc[middf.Month<3].sum())/(len(middf.T_degC.loc[middf.Year == 1949+x-1].loc[middf.Month>=12])+len(middf.T_degC.loc[middf.Year == 1949+x].loc[middf.Month<3]))
        else:
            Sal[YearF*x+y]=middf.Salnty.loc[middf.Year == 1949+x].loc[middf.Month>=(y+1)*YearT-1].loc[middf.Month<(y+2)*YearT-1].sum()/len(middf.T_degC.loc[middf.Year == 1949+x].loc[middf.Month>=(y+1)*YearT-1].loc[middf.Month<(y+2)*YearT-1])
            ST[YearF*x+y]=middf.STheta.loc[middf.Year == 1949+x].loc[middf.Month>=(y+1)*YearT-1].loc[middf.Month<(y+2)*YearT-1].sum()/len(middf.T_degC.loc[middf.Year == 1949+x].loc[middf.Month>=(y+1)*YearT-1].loc[middf.Month<(y+2)*YearT-1])
            Temp[YearF*x+y]=middf.T_degC.loc[middf.Year == 1949+x].loc[middf.Month>=(y+1)*YearT-1].loc[middf.Month<(y+2)*YearT-1].sum()/len(middf.T_degC.loc[middf.Year == 1949+x].loc[middf.Month>=(y+1)*YearT-1].loc[middf.Month<(y+2)*YearT-1])
middata=pd.DataFrame(np.column_stack((Yr,Ssn,Temp,Sal,ST)),columns=['Year','Season','T_degC','Salnty','STheta'])
middata=middata.drop(middata.index[0])

YearF=4
YearT=12/YearF
YearI=int(max(df.Year)-min(df.Year)+1)
Temp=[None]*YearI*YearF
Sal=[None]*YearI*YearF
ST=[None]*YearI*YearF
Yr=[None]*YearI*YearF
Ssn=[None]*YearI*YearF
for x in range(0,YearI):
    for y in range(0,YearF):
        Yr[YearF*x+y]=1949+x
        Ssn[YearF*x+y]=y+1
        if y == 0:
            Sal[YearF*x+y]=(highdf.Salnty.loc[highdf.Year == 1949+x-1].loc[highdf.Month>=12].sum()+highdf.Salnty.loc[highdf.Year == 1949+x].loc[highdf.Month<3].sum())/(len(highdf.T_degC.loc[highdf.Year == 1949+x-1].loc[highdf.Month>=12])+len(highdf.T_degC.loc[highdf.Year == 1949+x].loc[highdf.Month<3]))
            ST[YearF*x+y]=(highdf.STheta.loc[highdf.Year == 1949+x-1].loc[highdf.Month>=12].sum()+highdf.Salnty.loc[highdf.Year == 1949+x].loc[highdf.Month<3].sum())/(len(highdf.T_degC.loc[highdf.Year == 1949+x-1].loc[highdf.Month>=12])+len(highdf.T_degC.loc[highdf.Year == 1949+x].loc[highdf.Month<3]))
            Temp[YearF*x+y]=(highdf.Salnty.loc[highdf.Year == 1949+x-1].loc[highdf.Month>=12].sum()+highdf.Salnty.loc[highdf.Year == 1949+x].loc[highdf.Month<3].sum())/(len(highdf.T_degC.loc[highdf.Year == 1949+x-1].loc[highdf.Month>=12])+len(highdf.T_degC.loc[highdf.Year == 1949+x].loc[highdf.Month<3]))
        else:
            Sal[YearF*x+y]=highdf.Salnty.loc[highdf.Year == 1949+x].loc[highdf.Month>=(y+1)*YearT-1].loc[highdf.Month<(y+2)*YearT-1].sum()/len(highdf.T_degC.loc[highdf.Year == 1949+x].loc[highdf.Month>=(y+1)*YearT-1].loc[highdf.Month<(y+2)*YearT-1])
            ST[YearF*x+y]=highdf.STheta.loc[highdf.Year == 1949+x].loc[highdf.Month>=(y+1)*YearT-1].loc[highdf.Month<(y+2)*YearT-1].sum()/len(highdf.T_degC.loc[highdf.Year == 1949+x].loc[highdf.Month>=(y+1)*YearT-1].loc[highdf.Month<(y+2)*YearT-1])
            Temp[YearF*x+y]=highdf.T_degC.loc[highdf.Year == 1949+x].loc[highdf.Month>=(y+1)*YearT-1].loc[highdf.Month<(y+2)*YearT-1].sum()/len(highdf.T_degC.loc[highdf.Year == 1949+x].loc[highdf.Month>=(y+1)*YearT-1].loc[highdf.Month<(y+2)*YearT-1])
highdata=pd.DataFrame(np.column_stack((Yr,Ssn,Temp,Sal,ST)),columns=['Year','Season','T_degC','Salnty','STheta'])
highdata=highdata.drop(highdata.index[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_laun

In [ ]:
pd.set_option('display.max_rows', 10)
lowdata

,Year,Season,T_degC,Salnty,STheta
1,1949.0,2.0,14.742730,33.282691,24.669853
2,1949.0,3.0,16.879870,33.259035,24.166532
3,1949.0,4.0,15.908807,33.234647,24.372275
4,1950.0,1.0,33.354931,33.354931,33.354931
5,1950.0,2.0,13.984927,33.317026,24.863408
...,...,...,...,...,...
279,2018.0,4.0,NaN,NaN,NaN
280,2019.0,1.0,33.532331,33.532331,33.532331
281,2019.0,2.0,15.864524,33.551963,24.636188
282,2019.0,3.0,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_rows', 10)
middata

,Year,Season,T_degC,Salnty,STheta
1,1949.0,2.0,10.457389,33.548066,25.712890
2,1949.0,3.0,10.339411,33.531038,25.716322
3,1949.0,4.0,10.481904,33.497752,25.665846
4,1950.0,1.0,33.617823,33.617823,33.617823
5,1950.0,2.0,10.134502,33.635611,25.844010
...,...,...,...,...,...
279,2018.0,4.0,NaN,NaN,NaN
280,2019.0,1.0,33.703216,33.703216,33.703216
281,2019.0,2.0,10.914530,33.766584,25.816152
282,2019.0,3.0,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_rows', 10)
highdata

,Year,Season,T_degC,Salnty,STheta
1,1949.0,2.0,5.438834,34.258591,27.026743
2,1949.0,3.0,5.215942,34.282899,27.071624
3,1949.0,4.0,5.469926,34.252311,27.017963
4,1950.0,1.0,34.283299,34.283299,34.283299
5,1950.0,2.0,5.544914,34.299924,27.047712
...,...,...,...,...,...
279,2018.0,4.0,NaN,NaN,NaN
280,2019.0,1.0,34.184626,34.184626,34.184626
281,2019.0,2.0,7.284340,34.170502,26.730212
282,2019.0,3.0,NaN,NaN,NaN


Para poder encontrar los datos datos ausentes de la base se van a usar redes LSTM para cada datos ausente y de esta manera poder tener una base apta para la predicción de la temperatura, la salinidad y la densidad potencial del agua. 

In [ ]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

rangee=lowdata.T_degC.isnull().sum()
for xxxx in range(rangee):
    # define input sequence
    stop=lowdata.T_degC.isnull().idxmax()
    # Prepare data
    series = lowdata.loc[0:stop-1]
    in_seq1 = series.T_degC.values
    in_seq2 = series.Salnty.values
    out_seq = series.STheta.values
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    dataset = hstack((in_seq1, in_seq2, out_seq))
    n_steps = 3
    X, y = split_sequences(dataset, n_steps)
    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=150, verbose=0)
    # demonstrate prediction
    x_input = lowdata[['T_degC','Salnty','STheta']].loc[stop-n_steps:stop-1].values
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    lowdata.T_degC[stop]=yhat[0,0]
    lowdata.Salnty[stop]=yhat[0,1]
    lowdata.STheta[stop]=yhat[0,2]
    print("low data:",xxxx+1,"/",rangee)
rangee=middata.T_degC.isnull().sum()
for xxxx in range(rangee):
#Middata
    # define input sequence
    stop=middata.T_degC.isnull().idxmax()
    # Prepare data
    series = middata.loc[0:stop-1]
    in_seq1 = series.T_degC.values
    in_seq2 = series.Salnty.values
    out_seq = series.STheta.values
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    dataset = hstack((in_seq1, in_seq2, out_seq))
    n_steps = 3
    X, y = split_sequences(dataset, n_steps)
    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=150, verbose=0)
    # demonstrate prediction
    x_input = middata[['T_degC','Salnty','STheta']].loc[stop-n_steps:stop-1].values
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    middata.T_degC[stop]=yhat[0,0]
    middata.Salnty[stop]=yhat[0,1]
    middata.STheta[stop]=yhat[0,2]
    print("mid data:",xxxx+1,"/",rangee)
#Highdata
rangee=highdata.T_degC.isnull().sum()
for xxxx in range(rangee):
    #define input sequence
    stop=highdata.T_degC.isnull().idxmax()
    # Prepare data
    series = highdata.loc[0:stop-1]
    in_seq1 = series.T_degC.values
    in_seq2 = series.Salnty.values
    out_seq = series.STheta.values
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    dataset = hstack((in_seq1, in_seq2, out_seq))
    n_steps = 3
    X, y = split_sequences(dataset, n_steps)
    # the dataset knows the number of features, e.g. 2
    n_features = X.shape[2]
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=150, verbose=0)
    # demonstrate prediction
    x_input = highdata[['T_degC','Salnty','STheta']].loc[stop-n_steps:stop-1].values
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    highdata.T_degC[stop]=yhat[0,0]
    highdata.Salnty[stop]=yhat[0,1]
    highdata.STheta[stop]=yhat[0,2]
    print("high data:",xxxx+1,"/",rangee)


low data: 1 / 39
low data: 2 / 39
low data: 3 / 39
low data: 4 / 39
low data: 5 / 39
low data: 6 / 39
low data: 7 / 39
low data: 8 / 39
low data: 9 / 39
low data: 10 / 39
low data: 11 / 39
low data: 12 / 39
low data: 13 / 39
low data: 14 / 39
low data: 15 / 39
low data: 16 / 39
low data: 17 / 39
low data: 18 / 39
low data: 19 / 39
low data: 20 / 39
low data: 21 / 39
low data: 22 / 39
low data: 23 / 39
low data: 24 / 39
low data: 25 / 39
low data: 26 / 39
low data: 27 / 39
low data: 28 / 39
low data: 29 / 39
low data: 30 / 39
low data: 31 / 39
low data: 32 / 39
low data: 33 / 39
low data: 34 / 39
low data: 35 / 39
low data: 36 / 39
low data: 37 / 39
low data: 38 / 39
low data: 39 / 39
mid data: 1 / 39
mid data: 2 / 39
mid data: 3 / 39
mid data: 4 / 39
mid data: 5 / 39
mid data: 6 / 39
mid data: 7 / 39
mid data: 8 / 39
mid data: 9 / 39
mid data: 10 / 39
mid data: 11 / 39
mid data: 12 / 39
mid data: 13 / 39
mid data: 14 / 39
mid data: 15 / 39
mid data: 16 / 39
mid data: 17 / 39
mid data: 

Por último ya se escuentran los datos preprocesados y listo para implementarlos en la predicción de 3 años más apartir del último dato tomado. 

In [ ]:
pd.set_option('display.max_rows', 10)
lowdata

,Year,Season,T_degC,Salnty,STheta
1,1949.0,2.0,14.742730,33.282691,24.669853
2,1949.0,3.0,16.879870,33.259035,24.166532
3,1949.0,4.0,15.908807,33.234647,24.372275
4,1950.0,1.0,33.354931,33.354931,33.354931
5,1950.0,2.0,13.984927,33.317026,24.863408
...,...,...,...,...,...
163,1989.0,4.0,16.301615,33.447114,24.467104
164,1990.0,1.0,33.125843,33.246628,31.401169
165,1990.0,2.0,15.889104,33.511276,24.954891
166,1990.0,3.0,16.618359,33.457046,24.390780


In [ ]:
lowdata.to_csv (r'C:\Users\USER\Documents\low.csv', index = False, header=True)
middata.to_csv (r'C:\Users\USER\Documents\mid.csv', index = False, header=True)
highdata.to_csv (r'C:\Users\USER\Documents\high.csv', index = False, header=True)